In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import h5py
import cv2
import os
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset, random_split
import requests
import torch.optim as optim
import pandas as pd
from tqdm import tqdm

In [ ]:
label = pd.read_csv('san_diego_2024-01-01_to_2024-03-09.csv')

In [ ]:
y = label['temp'][:48]
y = torch.tensor(y)

In [ ]:
# Function to download image from URL and save to folder
def download_image(url, folder):
    # Get the filename from the URL
    filename = url.split('/')[-1]

    # Create the folder if it doesn't exist
    os.makedirs(folder, exist_ok=True)

    # Download the image and save it to the folder
    with open(os.path.join(folder, filename), 'wb') as f:
        response = requests.get(url)
        f.write(response.content)
    print(f"Downloaded {filename}")

# Read the text file containing image URLs
file_path = 'L2_LST_images.txt'  # Replace with the path to your text file
with open(file_path, 'r') as file:
    urls = file.readlines()

# Remove whitespace characters like `\n` at the end of each line
urls = [url.strip() for url in urls]

# Specify the folder to save the downloaded images
output_folder = 'L2_LST_images'

# Download images and save them to the folder
for url in urls:
    download_image(url, output_folder)

print("All images downloaded and saved to", output_folder)

Downloaded ECOSTRESS_L2_LSTE_32159_012_20240308T125756_0601_01.1.jpg
Downloaded ECOSTRESS_L2_LSTE_32154_001_20240308T044928_0601_01.1.jpg
Downloaded ECOSTRESS_L2_LSTE_32139_004_20240307T053804_0601_01.1.jpg
Downloaded ECOSTRESS_L2_LSTE_32093_004_20240304T062514_0601_01.1.jpg
Downloaded ECOSTRESS_L2_LSTE_32078_004_20240303T071319_0601_02.1.jpg
Downloaded ECOSTRESS_L2_LSTE_32052_007_20240301T152143_0601_01.1.jpg
Downloaded ECOSTRESS_L2_LSTE_32037_015_20240229T160915_0601_02.1.jpg
Downloaded ECOSTRESS_L2_LSTE_32032_003_20240229T080014_0601_01.1.jpg
Downloaded ECOSTRESS_L2_LSTE_32017_004_20240228T084843_0601_01.1.jpg
Downloaded ECOSTRESS_L2_LSTE_32017_003_20240228T084751_0601_01.1.jpg
Downloaded ECOSTRESS_L2_LSTE_31991_014_20240226T165559_0601_01.1.jpg
Downloaded ECOSTRESS_L2_LSTE_31971_003_20240225T093440_0601_01.1.jpg
Downloaded ECOSTRESS_L2_LSTE_31956_004_20240224T102239_0601_01.1.jpg
Downloaded ECOSTRESS_L2_LSTE_31930_015_20240222T183142_0601_01.1.jpg
Downloaded ECOSTRESS_L2_LSTE_31915

In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images
images = load_images_from_folder(output_folder)

resized_images = []
for img in images:
    resized_img = cv2.resize(img, (128, 128))
    resized_images.append(resized_img)

numpy_images = np.array(resized_images)

# Convert numpy array to PyTorch tensor
tensor_images = torch.from_numpy(numpy_images)

print("Shape of tensor_images:", tensor_images.shape)

Shape of tensor_images: torch.Size([49, 128, 128, 3])


In [ ]:
# Batch size: 4, Channels: 1 (for grayscale images, or it could be 3 for RGB if the data is like that), height: 32, width: 32
class CRNN(nn.Module):
    def __init__(self):
        super(CRNN, self).__init__()

        # CNN layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3, 3))
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3))
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3))
        self.pool3 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        # Flatten layer
        self.flatten = nn.Flatten()

        # LSTM layer
        # Assuming the flattened output is considered as sequ, batch_first=True)
ential data for LSTM
        self.lstm = nn.LSTM(input_size=1024, hidden_size=1024
        # Dense layer
        self.fc = nn.Linear(1024, 512)
        self.fc1 = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, 1) # output size is the product of height and width of the target map

    def forward(self, x):
        # Pass data through convolutional layers
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = self.pool3(torch.relu(self.conv3(x)))

        # Flatten the output for the LSTM
        x = self.flatten(x)

        # Reshape data for LSTM layer (batch_size, seq_len, features)
        x = x.view(4, -1, 256)

        # LSTM layer expects input as (batch, seq, feature)
        lstm_out, (hn, cn) = self.lstm(x)

        # Only take the output of the last time step (assuming we only predict 1 future step)
        # If we want to predict more steps, we should modify this part accordingly
        last_time_step = lstm_out[:, -1, :]

        # Pass the output of the last time step through the dense layer to get predictions
        x = self.fc(last_time_step)
        x = F.relu(x)
        x = self.fc1(x)
        x =F.relu(x)
        x = self.fc2(x)

        # Reshape the output to the shape of the target map
        # x = x.view(4, 36, 36)

        return x

# Create the CRNN model
crnn = CRNN()

# Print the model summary
print(crnn)


CRNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (lstm): LSTM(1024, 1024, batch_first=True)
  (fc): Linear(in_features=1024, out_features=512, bias=True)
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=1, bias=True)
)


In [ ]:
images = tensor_images.permute(0,3,2,1)
# Resize the images to 32x32 and reorder dimensions to [batch_size, channels, height, width]
transform = transforms.Compose([
    transforms.ToPILImage(),  # Convert numpy images to PIL Image to use torchvision transforms
    transforms.Resize((32, 32)),  # Resize image to 32x32
    transforms.ToTensor()  # Convert PIL Image to Tensor and reorder dimensions to [C, H, W]
])

images_transformed = torch.stack([transform(img) for img in images])

# Since your dataset size might not be a multiple of your batch size,
# let's truncate the dataset to the largest multiple of 4 for simplicity.
batch_size = 4
num_batches = len(images_transformed) // batch_size
images_transformed = images_transformed[:num_batches * batch_size]

# Create DataLoader
dataset = TensorDataset(images_transformed, y)

# Calculate sizes for splitting
test_size = int(len(dataset) * 0.2)  # Define the proportion for the test set, e.g., 20%
train_size = len(dataset) - test_size  # The rest will be used for the training set

# Split the dataset
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for both datasets
batch_size = 4  # Adjust based on your requirements
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Create the CRNN model
crnn = CRNN()

crnn.train()

criterion = nn.MSELoss()
optimizer = optim.Adam(crnn.parameters(), lr=0.001)

# Number of epochs (iterations over the entire training dataset)
epochs = 10

# Training loop
for epoch in tqdm(range(epochs)):
    running_loss = 0.0
    for data, label in train_loader:

        inputs = data[0]
        inputs = inputs.float()  # Ensure inputs are float
        label = label.float().unsqueeze(1)
        optimizer.zero_grad()
        outputs = crnn(inputs)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # Print every 10 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

print('Finished Training')

100%|██████████| 10/10 [00:06<00:00,  1.43it/s]

Finished Training


In [ ]:
crnn.eval()
with torch.no_grad():
    test_loss = 0.0
    for data, label in test_loader:
        inputs = data[0]
        outputs = crnn(inputs)
        loss = criterion(outputs, label)
        test_loss += loss.item()

    avg_test_loss = test_loss / len(test_loader)
    print(f'Average test loss: {avg_test_loss:.3f}')

Average test loss: 2.913


In [ ]:
# Redefine the CRNN model with GRU instead of LSTM

class CRNN_GRU(nn.Module):
    def __init__(self):
        super(CRNN_GRU, self).__init__()

        # CNN layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(3, 3))
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3))
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3))
        self.pool3 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        # Flatten layer
        self.flatten = nn.Flatten()

        # GRU layer
        self.gru = nn.GRU(input_size=3072, hidden_size=1024, batch_first=True)

        # Dense layer
        self.fc = nn.Linear(1024, 36*62) # output size is the product of height and width of the target map

    def forward(self, x):
        # Pass data through convolutional layers
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = self.pool3(torch.relu(self.conv3(x)))

        # Flatten the output for the GRU
        x = self.flatten(x)

        # Reshape data for GRU layer (batch_size, seq_len, features)
        x = x.view(4, -1, 3072)

        # GRU layer expects input as (batch, seq, feature)
        gru_out, hn = self.gru(x)

        # Only take the output of the last time step (assuming we only predict 1 future step)
        last_time_step = gru_out[:, -1, :]

        # Pass the output of the last time step through the dense layer to get predictions
        x = self.fc(last_time_step)

        # Reshape the output to the shape of the target map
        x = x.view(4, 36, 62)

        return x

# Create the CRNN model with GRU
crnn_gru = CRNN_GRU()

# Print the model summary
print(crnn_gru)
